# Case 2 - Tranfer model
Neural Networks for Machine Learning Applications<br>
24.2.2022, Jesse Jyrälä & Lauri Marjanen<br>
Metropolia University of Applied Sciences

# Idea of this model

In [1]:
#Imports
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

print('Tensorflow version: ', tf.__version__)

Tensorflow version:  2.7.0


# Dataset

Fetching data and splitting it into training and validation data. 4 to 1 split seems to work best but 10 to 1 could also be used with good results.

In [2]:
BATCH_SIZE = 32
IMG_SIZE = (128, 128)

In [3]:
data_dir = "D:/Koulu/Datasets/Case2Data/chest_xray/train"
os.listdir(data_dir)

['NORMAL', 'PNEUMONIA']

In [4]:
# Creating datasets for training and validation with 4 to 1 split
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split = 0.2,
  subset = "training",
  seed = 123,
  image_size = IMG_SIZE,
  batch_size = BATCH_SIZE)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split = 0.2,
  subset = "validation",
  seed = 123,
  image_size = IMG_SIZE,
  batch_size = BATCH_SIZE)

Found 5216 files belonging to 2 classes.
Using 4173 files for training.
Found 5216 files belonging to 2 classes.
Using 1043 files for validation.


# Modeling

First using keras pretarained models to base train the model. Testing ResNet50 and ResNet101

In [5]:
IMG_SHAPE = IMG_SIZE + (3,)

!!Remeber to comment out all pretrained models but the one you want to use!!

In [6]:
#pretrained model with ResNet101

preprocess_input = tf.keras.applications.resnet.preprocess_input

base_model = tf.keras.applications.ResNet101(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

171458560/171446536 [==============================] - 5s 0us/step


In [7]:
#pretrained model with ResNet50

# preprocess_input = tf.keras.applications.resnet.preprocess_input

# base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')

# base_model.trainable = False

In [8]:
#getting layers from keras for the child model to use

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

In [9]:
#Modeling the child

inputs = tf.keras.Input(shape = IMG_SHAPE)
x = preprocess_input(inputs)
x = base_model(x, training = False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [10]:
#compiling model. EDIT OUT: ei mitään hajuu mitä toi leraning rate tekee

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 128, 128, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 128, 128, 3)      0         
                                                                 
 resnet101 (Functional)      (None, 4, 4, 2048)        42658176  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0     

In [12]:
model.trainable_variables

[<tf.Variable 'dense/kernel:0' shape=(2048, 1) dtype=float32, numpy=
 array([[-0.03328813],
        [ 0.04641838],
        [-0.00106713],
        ...,
        [-0.03449555],
        [-0.01333673],
        [-0.01816825]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

# Training

In [13]:
loss1, accuracy1 = model.evaluate(val_dataset)

print(f"Validation loss:     {loss1:.2f}")
print(f"Validation accuracy: {accuracy1:.2f}")

33/33 [==============================] - 41s 1s/step - loss: 0.8915 - accuracy: 0.4295
Validation loss:     0.89
Validation accuracy: 0.43


In [14]:
trained = model.fit(train_dataset, batch_size=32, epochs=5)

Epoch 1/5
131/131 [==============================] - 158s 1s/step - loss: 0.5944 - accuracy: 0.7103
Epoch 2/5
131/131 [==============================] - 155s 1s/step - loss: 0.3004 - accuracy: 0.8567
Epoch 3/5
131/131 [==============================] - 153s 1s/step - loss: 0.2332 - accuracy: 0.8994
Epoch 4/5
131/131 [==============================] - 154s 1s/step - loss: 0.2050 - accuracy: 0.9101
Epoch 5/5
131/131 [==============================] - 155s 1s/step - loss: 0.1877 - accuracy: 0.9207


# Evaluation

In [15]:
loss1, accuracy1 = model.evaluate(val_dataset)

print(f"Validation loss:     {loss1:.2f}")
print(f"Validation accuracy: {accuracy1:.2f}")

33/33 [==============================] - 40s 1s/step - loss: 0.1199 - accuracy: 0.9521
Validation loss:     0.12
Validation accuracy: 0.95


In [16]:

acc = trained.history['accuracy']
val_acc = trained.history['val_accuracy']

loss = trained.history['loss']
val_loss = trained.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')
plt.grid()

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.grid()
plt.show()

KeyError: 'val_accuracy'

# Conclusion